In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad, dblquad
from scipy.constants import physical_constants, e,  epsilon_0, hbar, m_e, pi

The normalized 1s hydrogen atomic orbital is $$\psi_{1s}(\mathbf{r})= \frac{1}{\sqrt{ \pi a_{0}^3 }}e^{-r/a_{0}}$$
- $a_{0}$ is the bohr radius
- We place the atoms at $\mathbf{r}=-\frac{R}{2}\hat{z}$ and $\mathbf{r}=\frac{R}{2}\hat{z}$

The $$\psi_{1}(\mathbf{r})=\psi_{1s}\left( \left\lvert  \mathbf{r}+\frac{R}{2}\hat{z}   \right\rvert  \right)$$
and $$\psi_{2}(\mathbf{r})=\psi_{1s}\left( \left\lvert  \mathbf{r}-\frac{R}{2}\hat{z}   \right\rvert  \right)$$


## Step 2: Using the Scrhodinger Function
Now we know the time indepedent Schrodinger equation is of form: $$\hat{H}\Psi(\vec{r})=E\Psi(\vec{r})$$
Now we use the basis functions as the 1s orbitals of the hydrogen atoms.
So we can write our overall wavefunction of the molecule as the superposition of them: $$\mathbf{\Psi}=c_{1}\psi_{1}+c_{2}\psi_{2}$$
Hence we can write $$\hat{H}(c_{1}\ket{\psi_{1}}+c_{2}\ket{\psi_{2}}  )=E(c_{1}\ket{\psi_{1}}+c_{2}\ket{\psi_{2}}  )$$
$$c_{1}\hat{H}\ket{\psi_{1}}+c_{2}\hat{H}\ket{\psi_{2}} =c_{1}E\ket{\psi_{1}}+c_{2}E\ket{\psi_{2}}   $$
Now we bring we find the $H_{mn}$ elements, effectively finding the contribution of $\psi_{1},\psi_{2}$ to new Hamiltonian. $$c_{1}\braket{ \psi_{1} |\hat{H}|\psi_{1}  }+c_{2}\braket{ \psi_{1}|\hat{H}|\psi_{2}  }=E(c_{1}\braket{ \psi_{1} |\psi_{1}  }+c_{2}\braket{ \psi_{1}|\psi_{2}  })   $$
Now we write: 
$$\braket{ \psi_{1}|\hat{H}|\psi_{1}  }=\epsilon $$
$$\braket{ \psi_{1}|\hat{H}|\psi_{2}}=t$$ 
$$\braket{ \psi_{2}|\hat{H}|\psi_{1}}=t$$
$$\braket{ \psi_{2}|\hat{H}|\psi_{2}  }=\epsilon $$


But why $H_{11}=H_{22}~;~H_{12}=H_{21}$ ?
This is maybe due to the symmetry of the said molecule. The energy contribution of either atom is same due to symmetry. And so is the coupled energy.
Similarly: $$\braket{ \psi_{1} | \psi_{2} }=\braket{ \psi_{2} |\psi_{1}  } =s  $$

$$\braket{ \psi_{1} |\psi_{1}  }=\braket{ \psi_{2} |\psi_{2}  } =1  $$

From this we get our $S$ matrix. Note that I here wrote general representation. That is, that basis functions need not to be always orthonormal ($\braket{ \psi_{i} |\psi_{j}  }=0$). Applying our results we get following matrix formulation.

$$E\mathbf{S\Psi}=\mathbf{\hat{H}\Psi}$$

$$E\begin{pmatrix}
1 & s \\
s & 1
\end{pmatrix}\begin{pmatrix}
\psi_{1} \\
\psi_{2}
\end{pmatrix}=\begin{pmatrix}
\epsilon & t \\
t & \epsilon
\end{pmatrix}\begin{pmatrix}
\psi_{1} \\
\psi_{2}
\end{pmatrix}$$

## Step 3: Definitions of Integrals or Inner Products

- Overlap Integral:
$$s=\int \psi_{1}(r)\psi_{2}(r)d^3r$$
- On-site Energy
$$\epsilon=\int \psi_{1}^*(\mathbf{r})\left( -\frac{\hbar^2}{2m}-\frac{e^2}{4\pi\epsilon_{0}\left\lvert  \mathbf{r}+\frac{R}{2}  \right\rvert }-\frac{e^2}{4\pi\epsilon_{0}\left\lvert  \mathbf{r}-\frac{R}{2}  \right\rvert } \right)~\psi_{1}(\mathbf{r})~d^3r$$
- Hopping Integral
$$t=\int \psi_{1}^*(\mathbf{r})\hat{H}\psi_{2}(\mathbf{r})~d^3r$$



In [2]:
# Constants
a0=physical_constants['Bohr radius'][0]  # Bohr radius in meters
eh=physical_constants['Hartree energy'][0]  # Hartree energy in Joules
h2m=(hbar**2)/(2*m_e)  # h^2/(2m) in Joules

# Set internuclear distance R (in Bohr units)
R = 1.4  # ~equilibrium bond length of H2

# Hartree energy: is a unit of energy commonly used in atomic and molecular physics and quantum chemistry. 
# It is defined as the electrostatic potential energy of two electrons separated by the 
# Bohr radius in a vacuum. 

In [3]:
# 1s hydrogen orbital
def psi_1s(r):
    return (1 / np.sqrt(np.pi)) * np.exp(-r )

## Solving the Integrals

#### Geometry of the Problem

We are using a **simplified 1D radial approximation**, not full 3D, just to get intuition.

Suppose:

- Atom $1$ is centered at $z=0$
    
- Atom $2$ is at $z=R$
    
- $\psi_1(r) = \psi_{1s}(r)$, centered at $0$
    
- $\psi_2(r) = \psi_{1s}(|r - R|)$, centered at $R$

So the integrand becomes
$$\psi_{1}(r)\psi_{2}(r)=\psi_{1s}(r)\cdot\psi_{1s}(\lvert R-r \rvert )$$
Now the volume integral can be simplified for the spherical symmetry: $$\int d^3r=\int dr~r^2\cdot \int d\Omega$$
So we multiply by $r^2$ in the code, and also $\int d\Omega=4\pi$

In [58]:
# Defining the Integrals
# overlap integral S
def overlap_integral(R):
    integral = lambda r: psi_1s(r)*psi_1s(np.abs(R-r))*r**2
    s, _=quad(integral, 0, 20, limit=100)
    return s*4*np.pi

s=overlap_integral(R)
s


2.8059446536170722

`quad`: A function for numerical integration (quadrature) from the `scipy.integrate` module.

`integral`: The function to integrate.

`0, 20`: The lower and upper limits of integration.

`limits=100`:This sets the maximum number of subintervals for integration (used for handling difficult integrals).

`s, _`: The result of quad is a tuple: the first value (s) is the computed integral, the second (_) is the estimated error.

## Hopping Integral

Now the hopping integral is defined by the inner product as: $$t=\braket{ \psi|\hat{H}| \psi } $$
Now we have to write the Hamiltonian operator as: $$\hat{H}=\left( -\frac{h^2}{2m}\nabla^2 \right)+\left[ \left( -\frac{1}{r} \right)+\left( -\frac{1}{\lvert r-R \rvert } \right) \right]$$
Where: 
- $-\frac{h^2}{2m} \nabla^2$ is the **kinetic energy term**.
- **Potential from the nucleus at A**: $V_{A}=-\frac{1}{r}$
- **Potential from the nucleus at B**: $V_{B}=-\frac{1}{\lvert r-R \rvert}$

Thus, in atomic units, the total potential:
$$V(r)=-\frac{1}{r}-\frac{1}{\lvert r-R \rvert }$$
In our **simplified 1D radial approximation**, we ignore kinetic energy (we’ll revisit this later if needed), and only keep the **potential energy operator** part:

Hence we get: $$t=\int \psi_{1}(r)\cdot V(r)\cdot \psi_{2}(r)~d^3r$$



In [59]:
# Hopping Integral 
def hopping_integral(R):
    integral=lambda r: psi_1s(r)*(-1/r-1/np.abs(R-r))*psi_1s(np.abs(R-r))*r**2
    t, _ = quad(integral, 0, 20, limit=100)
    return t*4*np.pi

# Onsite energy
def onsite_energy():
    # For 1s in H atom: ⟨T⟩ = +13.6 eV, ⟨V⟩ = -27.2 eV, ⟨E⟩ = -13.6 eV
    return -1.0  # in Hartree (~ -13.6 eV)

t=hopping_integral(R)
t

C:\Users\kshit\AppData\Local\Temp\ipykernel_29100\1086991567.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  integral=lambda r: psi_1s(r)*(-1/r-1/np.abs(R-r))*psi_1s(np.abs(R-r))*r**2
C:\Users\kshit\AppData\Local\Temp\ipykernel_29100\1086991567.py:4: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  t, _ = quad(integral, 0, 20, limit=100)


-74.0833637477943

# Inconsistencies - Overlapping Integral .

The methodology and code we used gave wrong results for $s,t$. This is due to gross approximation we used by treating the entire problem as a 1D radial problem.

Now we will treat it as fully a 3D problem, by evaluating the **3D overlap integral** between two hydrogen 1s orbitals that are located at **two different positions** in space, specifically at:
- Atom A at $+R/2\hat{z}$ 
- Atom B at $-R/2\hat{z}$

Hence we write: $$s(R)=\int d^3\vec{r}~ \psi_{A}(\vec{r}) \psi_{B}(\vec{r})$$
where :
- $\psi_{A}(\vec{r})=\psi_{1s}(\vec{r}-\vec{R}/2)$
- $\psi_{B}(\vec{r})=\psi_{1s}(\vec{r}+\vec{R}/2)$

So this is a **volume integral over all space**, of the **product of two spherically symmetric orbitals**, each centered on different nuclei.

For this integral, we will shift to spherical coordinates due to spherical symmetry.
Hence we can write: $$d^3\vec{r}=r^2\sin \theta dr~d\theta ~d\phi$$
### The Function: `true_overlap_integral(R)`

#### 🔹 Inputs:

- `R`: distance between the two atoms
	
- `r1`: distance from point $\vec{r}$ to atom A (at $+R/2$ on z-axis)
    
- `r2`: distance from point $\vec{r}$ to atom B (at $−R/2$ on z-axis)

They are computed using the law of cosines: $$\left\lvert  \vec{r}-\frac{\vec{R}}{2}  \right\rvert =\sqrt{ r^2+R^2-2r(R/2)\cos \theta }$$
and similarly for the $\lvert \vec{r}+\vec{R}/2 \rvert$
#### 🔹 Next:

This is the **double integration** over $r$ and $θ$. `dblquad` handles nested integration:

- Outer: $r$ from $0$ to $10$ (good enough; orbitals vanish exponentially)
    
- Inner: $θ$ from $0$ to $π$

Finally we multiply the $2\pi$ due to : $\int_{0}^{2\pi}d\phi=2\pi$.


In [4]:
def full_overlap_integral(R):
    def integrand(theta, r):
        r1 = np.sqrt(r**2 + R**2 + 2*r*R*np.cos(theta))  # distance to nucleus A
        r2 = np.sqrt(r**2 + R**2 - 2*r*R*np.cos(theta))
        return psi_1s(r1) * psi_1s(r2) * np.sin(theta) * r**2

    result, _ = dblquad(
        func=integrand,
        a=0, b=3,                      # limit r to 10 Bohr radii
        gfun=lambda r: 0,
        hfun=lambda r: pi,
        epsabs=1e-10,
        epsrel=1e-10
    )
    return 2 * pi * result
s





NameError: name 's' is not defined

### Inconsistencies- Hopping Function

Now that we were computing $$t(R)=\int \psi_{A}^*(\vec{r})\cdot \hat{H}\cdot \psi(\vec{r})~d^3r$$
where we neglect the kinetic part in the Hamiltonian, to only give us the potential term.

$$t(R)\approx \int \psi_{A}^*(\vec{r})\cdot V_{Total}(\vec{r})\cdot \psi_{B}(\vec{r})d^3r$$

First we need the potential function, which we can get as: $$V(\vec{r})=-\frac{1}{\lvert \vec{r}-R/2 \rvert }-\frac{1}{\lvert \vec{r}+R/2 \rvert }$$



In [5]:

def full_overlap_integral(R):
    def integrand(theta, r):
        r1 = np.sqrt(r**2 + R**2 + 2*r*R*np.cos(theta))  # distance to nucleus A
        r2 = np.sqrt(r**2 + R**2 - 2*r*R*np.cos(theta))
        return psi_1s(r1) * psi_1s(r2) * np.sin(theta) * r**2

    result, _ = dblquad(
        func=integrand,
        a=0, b=1.7,                      # limit r to 10 Bohr radii
        gfun=lambda r: 0,
        hfun=lambda r: pi,
        epsabs=1e-10,
        epsrel=1e-10
    )
    return 2 * pi * result

def V_total(r, theta, R):
    rA = np.sqrt(r**2 + (R/2)**2 - r*R*np.cos(theta))  # distance to nucleus A
    rB = np.sqrt(r**2 + (R/2)**2 + r*R*np.cos(theta))  # distance to nucleus B
    return -1 / rA - 1 / rB

def t_integral(R):
    def integrand(theta, r):
        r1 = np.sqrt(r**2 + R**2 - 2*r*R*np.cos(theta))
        r2 = np.sqrt(r**2 + R**2 + 2*r*R*np.cos(theta))
        V = V_total(r, theta, R)
        return psi_1s(r1) * V * psi_1s(r2) * np.sin(theta) * r**2
    result, _ = dblquad(integrand, 0, np.inf, lambda r: 0, lambda r: np.pi)
    return 2 * np.pi * result


R = 1.4
s = full_overlap_integral(R)
t = t_integral(R)
epsilon = -1.0  # Hartree
s,t

(0.1907106992242572, -0.5323321889560118)

In [7]:
from scipy.integrate import quad
import numpy as np

def f(x):
    return np.exp(-x)

result, _ = quad(f, 0, 10)
print("Integral should be ≈ 1:", result)


Integral should be ≈ 1: 0.9999546000702375


In [6]:
R = 1.4
s = full_overlap_integral(R)
t = t_integral(R)
epsilon = -1.0  # Hartree
s

0.1907106992242572

### MO Theory Schrodinger Equation

Now that we have gotten our coefficients required for the Hamiltonian matrix, we can write: $$\hat{H} \mathbf{\Psi}=E\mathbf{\Psi}$$
And $$\hat{H}\psi=ES\psi$$
In terms of basis functions.

$$\begin{bmatrix}
\epsilon & t \\
t & \epsilon
\end{bmatrix}\begin{bmatrix}
\psi_{A} \\
\psi_{B}
\end{bmatrix}=E\begin{bmatrix}
1 & s \\
 s& 1
\end{bmatrix}\begin{bmatrix}
\psi_{A} \\
\psi_{B}
\end{bmatrix}$$
Now this has become the eigenvalue problem, so we can easily solve this,


In [8]:
main_diag=epsilon*np.ones(2)
off_diag=t*np.ones(1)
off_diag_s=s*np.ones(1)
H=np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)

S=np.diag(np.ones(2)) + np.diag(off_diag_s, k=1) + np.diag(off_diag_s, k=-1)



In [9]:
from scipy.linalg import eigh
# Solve the generalized eigenvalue problem
eigenvalues, eigenvectors = eigh(H, S)

print("Eigenvalues (Hartree):", eigenvalues)
print("Eigenvalues (eV):", eigenvalues * 27.2114)

Eigenvalues (Hartree): [-1.28690553 -0.5778747 ]
Eigenvalues (eV): [-35.01850126 -15.72477958]


0.3899947986829746


✅ True Overlap s = 0.752942  (Expected ≈ 0.198)


# Afterwards